### 1) Conectar db

In [1]:
import pyodbc
server = 'tcp:banco.tce.pi.gov.br' 
database = 'Sagres2019_RPT' 
cnxn = pyodbc.connect('Trusted_Connection=yes;DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database)
cursor = cnxn.cursor()

### 2) Executar consulta db

In [2]:
cursor = cnxn.cursor()
cursor.execute("""SELECT
   Compras.licitacoesweb.LICITACAO.ID as  ID_LICITACAO , Compras.licitacoesweb.LICITANTE.CNPJ as LICITANTE_CNPJ, Compras.licitacoesweb.LICITANTE.NOME as LICITANTE_NOME , Compras.licitacoesweb.LICITACAO.DATA_ABERTURA AS DATA_ABERTURA, Compras.licitacoesweb.LICITACAO.DATA_DIVULGACAO AS DATA_DIVUGACAO, Compras.licitacoesweb.LICITACAO.DATA_FINALIZACAO AS DATA_FINALIZACAO, Compras.licitacoesweb.LICITACAO.ID_UNIDADE_GESTORA AS UNIDADE_GESTORA ,IIF( ISNULL(Compras.licitacoesweb.ITEM.NUM_ITEM,'L')!='L',1,0) AS VENCEDOR_FLAG, Compras.licitacoesweb.ITEM.NUM_ITEM AS NUM_ITEM, Compras.licitacoesweb.ITEM.VALOR_UNITARIO AS VALOR_UNITARIO_ITEM,Compras.licitacoesweb.ITEM.QUANTIDADE AS QUANTIDADE_ITEM ,Compras.licitacoesweb.ITEM.VALOR_PREVISTO AS VALOR_PREVISTO_ITEM  ,Compras.licitacoesweb.LICITACAO.VALOR_TOTAL_HOMOLOGADO AS  VALOR_TOTAL_HOMOLOGADO, Compras.licitacoesweb.LICITACAO.VALOR_TOTAL_PREVISTO AS VALOR_TOTAL_PREVISTO
FROM
    Compras.licitacoesweb.LICITACAO
    FULL join Compras.licitacoesweb.PARTICIPACAO ON Compras.licitacoesweb.LICITACAO.ID = Compras.licitacoesweb.PARTICIPACAO.ID_LICITACAO
    FULL join Compras.licitacoesweb.LICITANTE ON Compras.licitacoesweb.PARTICIPACAO.ID_LICITANTE = Compras.licitacoesweb.LICITANTE.ID 
    FULL join Compras.licitacoesweb.LOTE ON Compras.licitacoesweb.LOTE.ID_PARTICIPACAO = Compras.licitacoesweb.PARTICIPACAO.ID
    AND Compras.licitacoesweb.LOTE.ID_LICITACAO = Compras.licitacoesweb.LICITACAO.ID 
    FULL join Compras.licitacoesweb.ITEM ON Compras.licitacoesweb.ITEM.ID_LOTE = Compras.licitacoesweb.LOTE.ID
    FULL join Compras.licitacoesweb.TIPO_STATUS_LICITACAO ON Compras.licitacoesweb.TIPO_STATUS_LICITACAO.ID = Compras.licitacoesweb.LICITACAO.ID_STATUS_LICITACAO 
    FULL join Compras.licitacoesweb.TIPO_OBJETO on Compras.licitacoesweb.TIPO_OBJETO.ID = Compras.licitacoesweb.LICITACAO.ID_TIPO_OBJETO 
WHERE
    Compras.licitacoesweb.LICITACAO.DATA_ABERTURA BETWEEN '01/01/2019' AND '05/05/2020'
    /*
    and Compras.licitacoesweb.LICITACAO.DESCRICAO_OBJETO LIKE '%paralele%'
    */
    and Compras.licitacoesweb.LICITACAO.ID_STATUS_LICITACAO=3
    and Compras.licitacoesweb.TIPO_OBJETO.ID =1
 ORDER BY 
 ID_LICITACAO""") ;

### 3) Salvar Tabela e conv Panda

In [3]:
import pandas as pd
import numpy as ny
columns = [col[0] for col in cursor.description]
rows = [dict(zip(columns, row)) for row in cursor.fetchall()]

In [4]:
licitacoes= pd.DataFrame(rows)

In [5]:
licitacoes.head(5)

,ID_LICITACAO,LICITANTE_CNPJ,LICITANTE_NOME,DATA_ABERTURA,DATA_DIVUGACAO,DATA_FINALIZACAO,UNIDADE_GESTORA,VENCEDOR_FLAG,NUM_ITEM,VALOR_UNITARIO_ITEM,QUANTIDADE_ITEM,VALOR_PREVISTO_ITEM,VALOR_TOTAL_HOMOLOGADO,VALOR_TOTAL_PREVISTO
0,192813,02725914000145,Cerrado Engenharia Incorporadora Eirelli,2019-02-13 09:00:00,NaT,2019-05-02 14:18:15.422,1530,1,pg,243839.86,1.00,None,243839.86,244668.74
1,205722,04603664000104,V. M. PESSOA MONTEIRO-ME,2019-01-08 09:00:00,NaT,2019-08-19 12:12:38.690,149,1,pg,211598.50,1.00,None,211598.50,302118.82
2,205722,04521201000195,Construtora F.M Ltda. EPP,2019-01-08 09:00:00,NaT,2019-08-19 12:12:38.690,149,0,None,None,None,None,211598.50,302118.82
3,205722,06224118000180,Construtora Padrão Ltda. EPP,2019-01-08 09:00:00,NaT,2019-08-19 12:12:38.690,149,0,None,None,None,None,211598.50,302118.82
4,205722,29750013000160,GT CONSTRUTORA-E PEREIRA EIRELI EPP,2019-01-08 09:00:00,NaT,2019-08-19 12:12:38.690,149,0,None,None,None,None,211598.50,302118.82


### 4) Analise-vencedores x perdedores

In [6]:
tabela=licitacoes[['UNIDADE_GESTORA','LICITANTE_CNPJ','VENCEDOR_FLAG','ID_LICITACAO','VALOR_UNITARIO_ITEM']]

In [7]:
k=tabela.groupby(['LICITANTE_CNPJ']).agg({'ID_LICITACAO': 'count','VENCEDOR_FLAG': 'sum','VALOR_UNITARIO_ITEM': 'sum'})

In [8]:
k.sort_values(by=['ID_LICITACAO'],ascending=False)

,ID_LICITACAO,VENCEDOR_FLAG,VALOR_UNITARIO_ITEM
LICITANTE_CNPJ,,,
04603664000104,115,41,11569042.02
06224118000180,110,31,19082571.43
12187423000141,97,3,4503088.11
04521201000195,75,2,1808357.12
27055754000103,66,4,1348014.60
...,...,...,...
16099194000164,1,0,0
15689051000140,1,1,216667.66
15563861000155,1,0,0


### 4.1) Analise- Perdedores (filtro)

In [9]:
filter = k["VENCEDOR_FLAG"]<10
k=k.where(filter) 

In [10]:
k.sort_values(by=['ID_LICITACAO'],ascending=False)

,ID_LICITACAO,VENCEDOR_FLAG,VALOR_UNITARIO_ITEM
LICITANTE_CNPJ,,,
12187423000141,97.0,3.0,4503088.11
04521201000195,75.0,2.0,1808357.12
27055754000103,66.0,4.0,1348014.60
02412944000100,53.0,1.0,147304.05
12190146000126,52.0,7.0,1377025.28
...,...,...,...
28341992000130,NaN,NaN,NaN
28395450000140,NaN,NaN,NaN
28395623000120,NaN,NaN,NaN


### 4.1) Analise- Vencedores (filtro)

In [11]:
k=tabela.groupby(['LICITANTE_CNPJ']).agg({'ID_LICITACAO': 'count','VENCEDOR_FLAG': 'sum','VALOR_UNITARIO_ITEM': 'sum'})

In [12]:
filter1 = k["VENCEDOR_FLAG"]==k["ID_LICITACAO"]
filter2 = k["VENCEDOR_FLAG"]+10>=k["ID_LICITACAO"]

k=k.where(filter1 | filter2) 

In [13]:
k.sort_values(by=['ID_LICITACAO'],ascending=False)

,ID_LICITACAO,VENCEDOR_FLAG,VALOR_UNITARIO_ITEM
LICITANTE_CNPJ,,,
02733213000158,61.0,60.0,715574.88
29654913000103,61.0,53.0,606587.42
12070635000144,41.0,37.0,219000.26
21071556000149,37.0,32.0,3433845.79
22555169000140,37.0,29.0,452082.64
...,...,...,...
34966820000154,NaN,NaN,NaN
35139286000175,NaN,NaN,NaN
41506072000192,NaN,NaN,NaN


### 5) Analise-Combinações

In [150]:
from itertools import combinations
from collections import Counter

In [186]:
k=tabela.groupby(['UNIDADE_GESTORA','ID_LICITACAO']).agg({'LICITANTE_CNPJ':pd.Series.nunique})

In [187]:
l=tabela.groupby(['UNIDADE_GESTORA','ID_LICITACAO']).agg({'LICITANTE_CNPJ':'unique'})

In [188]:
tb=l.merge(k,right_on=['ID_LICITACAO','UNIDADE_GESTORA'], left_on=['ID_LICITACAO','UNIDADE_GESTORA'])

In [189]:
tb.rename(columns={'LICITANTE_CNPJ_y':'N_LICITANTES'}, inplace=True)

In [191]:
tb

,,LICITANTE_CNPJ_x,N_LICITANTES
ID_LICITACAO,UNIDADE_GESTORA,,
276751,1,"[07118909000198, 07239237000179, 0030033900010...",11
206099,3,[29750013000160],1
206788,3,[19783564000176],1
207219,3,[13466507000187],1
231954,3,[07467238000170],1
...,...,...,...
308878,2137,"[17323084000105, 11417820000108]",2
308885,2137,[17323084000105],1
319691,2137,"[24667970000103, 11417820000108]",2


In [34]:
products_list =  k.values.tolist()

count= Counter()

In [35]:
for licitacao in products_list :
    duplas =combinations(licitacao[0],2)
    count.update(Counter(duplas))

In [36]:
count=count.most_common()

In [37]:
count=dict(count)

In [38]:
for key in count.keys():
    if(count[key]>3):
        print("Dupla {0} apareceu {1} vezes".format(key,count[key]))

Dupla ('04521201000195', '06224118000180') apareceu 28 vezes
Dupla ('27055754000103', '04521201000195') apareceu 26 vezes
Dupla ('27390755000104', '12187423000141') apareceu 26 vezes
Dupla ('12187423000141', '04521201000195') apareceu 26 vezes
Dupla ('27055754000103', '06224118000180') apareceu 25 vezes
Dupla ('04521201000195', '12187423000141') apareceu 24 vezes
Dupla ('12187423000141', '02412944000100') apareceu 24 vezes
Dupla ('12187423000141', '07467238000170') apareceu 22 vezes
Dupla ('06224118000180', '12187423000141') apareceu 21 vezes
Dupla ('27055754000103', '12187423000141') apareceu 21 vezes
Dupla ('19783564000176', '04521201000195') apareceu 21 vezes
Dupla ('02412944000100', '12187423000141') apareceu 21 vezes
Dupla ('12187423000141', '27055754000103') apareceu 20 vezes
Dupla ('12187423000141', '06224118000180') apareceu 20 vezes
Dupla ('04521201000195', '27055754000103') apareceu 20 vezes
Dupla ('12190146000126', '12187423000141') apareceu 20 vezes
Dupla ('06224118000180',